In [1]:
from Neuro_Plotting.plot import plot_surf_parc, plot_surf
import numpy as np
import os
import matplotlib.pyplot as plt
from nilearn.surface import load_surf_data
import imageio
from joblib import Parallel, delayed

In [2]:
def _save_prob_plot(parc, i):
    
    print(i)
    
    # Load then delete for memory
    parcel = np.load(parc)
    chunk = parcel[:, i]
    parcel_shape = parcel.shape[1]
    del parcel
    
    # Gen title
    title = clean_name(parc) + f' ({i}/{parcel_shape})'

    # Plot and save to temp folder
    plot_surf(chunk,
              threshold=0.000001,
              cmap='Reds',
              title=title)

    temp_save_loc = 'temp/' + str(i) + '.png'
    plt.savefig(temp_save_loc, dpi=100,
                transparent=True, bbox_inches='tight')
    plt.close()
    
    # Return temp save_loc
    return temp_save_loc

def clean_name(parc):
    base = parc.split('/')[-1]
    name = base.replace('_', ' ').replace('.npy', '')
    name = name.replace('-', ' ')
    return name

def plot_prob_parc(parc):
    
    # Only proc. if not already saved
    save_loc = 'Figures/prob_parcels/' + parc.split('/')[-1].replace('.npy', '.gif')
    if os.path.exists(save_loc):
        return
    
    # Load
    parcel = np.load(parc)
    
    # Multi process save prob plots
    files = Parallel(n_jobs=4)(delayed(_save_prob_plot)(parc, i) for i in range(parcel.shape[1]))
    
    # Load files from temp and make gif
    with imageio.get_writer(save_loc,
                            mode='I',
                            duration=.5) as writer:

        for filename in files:
            image = imageio.imread(filename)
            writer.append_data(image)
            
def plot_parc(parc):
    
    # Only make and save if doesn't exist
    save_loc = 'Figures/parcels/' + parc.split('/')[-1].replace('.npy', '.png')
    if os.path.exists(save_loc):
        return

    # Load
    parcel = np.load(parc)
    
    # Plot and save
    plot_surf_parc(parcel, title=clean_name(parc))
    plt.savefig(save_loc, dpi=100, transparent=True, bbox_inches='tight')
    plt.close()
    
# Main func
def plot_any_parc(parc):
    
    # Seperate procedure if prob.
    if '_prob' in parc:
        plot_prob_parc(parc)
        
    # Otherwise just plot and save
    else:
        plot_parc(parc)

In [3]:
# Get all parcels
parcs = os.listdir('../parcels/')
parcs = [os.path.join('../parcels/', p) for p in parcs]

# Make sure directories exist
os.makedirs('Figures/parcels/', exist_ok=True)
os.makedirs('Figures/prob_parcels/', exist_ok=True)
os.makedirs('temp/', exist_ok=True)

# Plot all the parcellations w/ multi-proc
_ = Parallel(n_jobs=16)(delayed(plot_any_parc)(parc) for parc in parcs)